In [1]:
import numpy as np
import json
from os import listdir
from os.path import join
import pandas as pd
from kashgari.tasks.seq_labeling import BLSTMCRFModel
import datetime

Using TensorFlow backend.


In [2]:
path = 'Data/SortData'
file = listdir(path)
file.sort()
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [3]:
import jieba.posseg as pseg
import jieba

In [4]:
#load jieba dict
jieba.set_dictionary('Data/dict.txt.big')
jieba.load_userdict('Data/Mydict.txt')

Building prefix dict from /data/samba_shared/AIPRLab/Lai/High_Speed_Project/Data/dict.txt.big ...
Loading model from cache /tmp/jieba.ub0654bdec7f2e7020895ff59b2ea2981.cache
Loading model cost 1.623 seconds.
Prefix dict has been built succesfully.


In [5]:
#找商品名字
def findProduct(text,pred):
    products = []
    index = 0
    while index < len(pred) :
        if pred[index] == 'I-PRO' or pred[index] == 'B-PRO':
            begin = index
            while index< len(pred) and (pred[index] == 'I-PRO' or pred[index] == 'B-PRO'):
                index += 1
            end = index
            products.append(text[begin:end])
        index += 1
    return products

In [23]:
#找商品數量
def findQuantity(text,products):
    #將中文、英文的數字轉成阿拉伯數字
    chs_arabic_map = {u'零':0, u'一':1, u'二':2, u'兩':2 ,u'三':3, u'四':4,
             u'五':5, u'六':6, u'七':7, u'八':8, u'九':9,u'十':10,u'one':1,u'two':2,u'three':3,u'four':4,u'five':5,
             u'six':6,u'seven':7,u'eight':8,u'nine':9}
    Quantity = []
    if len(products)!=0:
        for p in products:
            begin = text.find(p)
            end = begin+len(p)
            words = pseg.cut(text[end:])
            find = False
            for w in words :
                if w.flag == 'num': #如果是自己設定的詞性就是 +1,+2
                    Quantity.append( int(w.word[1:]) )
                    find = True
                    break
                elif w.flag=='eng': #可能是英文的數字
                    try:
                        Quantity.append(chs_arabic_map[w.word])
                        find = True
                        break
                    except:
                        continue
            if(not find): #找阿拉伯數字
                for w in jieba.cut(text[end:]):
                    if w.isdigit():
                        Quantity.append(int(w))
                        find = True
                        break
            if(not find): #找中文的數字
                for w in text[end:]:
                    if w.isnumeric() and (not w.isdigit()):
                        try:
                            Quantity.append(chs_arabic_map[w])
                            find = True
                            break
                        except:
                            continue
            if(not find):#如果所有可能都失敗 設數量為1
                Quantity.append(1)
                
    return Quantity          

In [8]:
def findnum(text): #找出沒商品名 只有打+1的
    words = pseg.cut(text)
    find = False
    num = 0
    for w in words :
        if w.flag == 'num':
            num= int(w.word[1:]) 
            find = True
            break
    return find,num

In [7]:
model = BLSTMCRFModel.load_model('Model')

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 100)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 100)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 100, 768), ( 16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 100, 768)     1536        Input-Segment[0][0]              
__________________________________________________________________________________________________
Embedding-

In [9]:
order= {'Username':[],'Product':[],'Quantity':[],'Date':[]}
for i in file:
    fullpath = join(path,i)
    message = pd.read_csv(fullpath)
    Preproduct = ''
    for user,text in zip(message['user'],message['text']):
        if user!='莊雅萍' and text == text: #篩掉店長的訊息
            pred = model.predict(text)
            products = findProduct(text,pred)
            
            if len(products)!=0: #如果有找到商品
                quantity = findQuantity(text,products)
                Preproduct = products[-1]
                for p,q in zip(products,quantity):
                    order['Product'].append(p)
                    order['Quantity'].append(q)
                    order['Date'].append(datetime.datetime.strptime(i[0:-4], '%Y-%m-%d').date())
                    order['Username'].append(user)
            elif Preproduct!='': #如果沒找到商品，判斷是不是只喊+1
                find,num = findnum(text)
                if find:
                    order['Product'].append(Preproduct)
                    order['Quantity'].append(num)
                    order['Date'].append(datetime.datetime.strptime(i[0:-4], '%Y-%m-%d').date())
                    order['Username'].append(user)
                

In [10]:
df = pd.DataFrame(order, columns = ['Username','Product','Quantity','Date'])
df.to_csv('Order.csv',index = False)

In [11]:
Orders = pd.read_csv('Order.csv')#將最後結果存成csv